📌 1. Definição do Problema de Predição Binária
🎯 Objetivo

Definir uma tarefa de classificação binária relacionada ao problema de negócio.
Neste caso, vamos prever se o estoque está em nível crítico (baixo) com base em variáveis operacionais.

In [5]:
import sklearn
print(sklearn.__version__)

1.7.2


In [6]:
import pandas as pd
import numpy as np

# Carregar dados (ajuste o caminho conforme necessário)
df = pd.read_excel("DasaMatHosp.xlsx")

# Garantir que DataHora é datetime
df['DataHora'] = pd.to_datetime(df['DataHora'], errors='coerce')

# Criar variável binária: estoque crítico
limite_critico = df['Estoque'].quantile(0.25)
df['EstoqueCritico'] = (df['Estoque'] < limite_critico).astype(int)

df.head()


,Material,DataHora,Estoque,Custo_Unitario,Custo_Total,EstoqueCritico
0,Seringas,2024-05-17 15:00:00,66.0,0.45,29.70,1
1,Máscaras N95,2024-05-27 09:00:00,320.0,5.15,1648.00,0
2,Algodão,2024-01-01 00:00:00,351.0,0.25,87.75,0
3,Seringas,2024-04-16 06:00:00,565.0,0.44,248.60,0
4,Luvas Cirúrgicas,2024-08-22 18:00:00,171.0,0.89,152.19,1


📊 2. Modelo KNN — Previsão Binária
🧩 Justificativa

O K-Nearest Neighbors (KNN) classifica com base na similaridade entre amostras.
O valor de k define o número de vizinhos.
Um bom ponto inicial é k = sqrt(n_amostras).

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Selecionar variáveis explicativas e alvo
df_modelo = df.dropna(subset=['Estoque', 'Custo_Unitario', 'Custo_Total', 'DataHora']).copy()
df_modelo = pd.get_dummies(df_modelo, columns=['Material'], drop_first=True)

X = df_modelo.drop(columns=['Estoque', 'DataHora', 'EstoqueCritico'])
y = df_modelo['EstoqueCritico']

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definição de k
k = int(np.sqrt(len(X_train)))
print("Valor de k escolhido:", k)

# Treinamento do modelo KNN
modelo_knn = KNeighborsClassifier(n_neighbors=k)
modelo_knn.fit(X_train_scaled, y_train)


Valor de k escolhido: 143


,n_neighbors,143
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


📈 3. Avaliação do Modelo KNN

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred_knn = modelo_knn.predict(X_test_scaled)

print("Acurácia:", accuracy_score(y_test, y_pred_knn))
print("Precisão:", precision_score(y_test, y_pred_knn))
print("Recall:", recall_score(y_test, y_pred_knn))
print("F1-Score:", f1_score(y_test, y_pred_knn))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_knn))


Acurácia: 0.9946493624772313
Precisão: 0.9900090826521344
Recall: 0.9886621315192744
F1-Score: 0.9893351486271841
Matriz de Confusão:
 [[6557   22]
 [  25 2180]]


🔢 4. Regressão Logística
🎯 Objetivo

Modelar a probabilidade de estoque crítico com um modelo linear probabilístico.

In [9]:
from sklearn.linear_model import LogisticRegression

modelo_log = LogisticRegression(max_iter=500)
modelo_log.fit(X_train_scaled, y_train)

y_pred_log = modelo_log.predict(X_test_scaled)


📊 Interpretação dos Coeficientes

In [10]:
coef_log = pd.DataFrame({
    'Variável': X.columns,
    'Coeficiente': modelo_log.coef_[0]
}).sort_values(by='Coeficiente', ascending=False)
coef_log.head(10)


,Variável,Coeficiente
0,Custo_Unitario,16.639520
6,Material_Máscaras N95,-0.724520
3,Material_Dipirona,-0.831967
7,Material_Paracetamol,-0.964194
9,Material_Soro Fisiológico,-1.338899
10,Material_Álcool 70%,-2.000262
5,Material_Luvas Cirúrgicas,-2.891969
8,Material_Seringas,-3.060235
2,Material_Algodão,-3.290164
4,Material_Gaze Estéril,-4.497152


⚖️ 5. Comparação — KNN vs Regressão Logística

In [11]:
metrics_comparison = pd.DataFrame({
    'Modelo': ['KNN', 'Regressão Logística'],
    'Acurácia': [
        accuracy_score(y_test, y_pred_knn),
        accuracy_score(y_test, y_pred_log)
    ],
    'Precisão': [
        precision_score(y_test, y_pred_knn),
        precision_score(y_test, y_pred_log)
    ],
    'Recall': [
        recall_score(y_test, y_pred_knn),
        recall_score(y_test, y_pred_log)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_knn),
        f1_score(y_test, y_pred_log)
    ]
})
metrics_comparison


,Modelo,Acurácia,Precisão,Recall,F1-Score
0,KNN,0.994649,0.990009,0.988662,0.989335
1,Regressão Logística,0.996699,0.990975,0.995918,0.993440


📉 6. Ridge Regression (Regressão Regularizada)
🎯 Objetivo

Aplicar regularização L2 para prever o Estoque (variável contínua).


In [12]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error

X_reg = df_modelo.drop(columns=['Estoque', 'DataHora', 'EstoqueCritico'])
y_reg = df_modelo['Estoque']

modelo_ridge = Ridge(alpha=1.0)
modelo_ridge.fit(X_reg, y_reg)

y_pred_ridge = modelo_ridge.predict(X_reg)

print("R² Ridge:", r2_score(y_reg, y_pred_ridge))
print("RMSE Ridge:", np.sqrt(mean_squared_error(y_reg, y_pred_ridge)))


R² Ridge: 0.635937098019246
RMSE Ridge: 94.30035436269263


🧮 7. Lasso Regression
🎯 Objetivo

Aplicar regularização L1 e verificar se variáveis irrelevantes são eliminadas.

In [13]:
from sklearn.linear_model import Lasso

modelo_lasso = Lasso(alpha=0.1)
modelo_lasso.fit(X_reg, y_reg)

coef_lasso = pd.DataFrame({
    'Variável': X_reg.columns,
    'Coef_Lasso': modelo_lasso.coef_
}).sort_values(by='Coef_Lasso', ascending=False)

print("Coeficientes não nulos:", (coef_lasso['Coef_Lasso'] != 0).sum())
coef_lasso.head(10)


Coeficientes não nulos: 8


,Variável,Coef_Lasso
3,Material_Dipirona,97.674394
7,Material_Paracetamol,84.387916
2,Material_Algodão,15.933697
10,Material_Álcool 70%,3.647857
1,Custo_Total,0.378500
9,Material_Soro Fisiológico,-0.000000
5,Material_Luvas Cirúrgicas,-0.000000
6,Material_Máscaras N95,-0.000000
4,Material_Gaze Estéril,-1.797271
8,Material_Seringas,-2.785889


📈 8. Regressão Polinomial
🎯 Objetivo

Testar um modelo de grau 2 para capturar relações não lineares.

In [14]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_reg)

modelo_poly = LinearRegression()
modelo_poly.fit(X_poly, y_reg)

y_pred_poly = modelo_poly.predict(X_poly)

print("R² Polinomial:", r2_score(y_reg, y_pred_poly))
print("RMSE Polinomial:", np.sqrt(mean_squared_error(y_reg, y_pred_poly)))


R² Polinomial: 0.9975007179915731
RMSE Polinomial: 7.813268391045277


🌳 9. Árvore de Decisão e Random Forest
🌲 Árvores de Decisão

In [15]:
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier(max_depth=4, random_state=42)
arvore.fit(X_train, y_train)
y_pred_arvore = arvore.predict(X_test)


🌳 Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

floresta = RandomForestClassifier(n_estimators=100, random_state=42)
floresta.fit(X_train, y_train)
y_pred_floresta = floresta.predict(X_test)


📊 Avaliação

In [17]:
for nome, y_pred in [("Árvore", y_pred_arvore), ("Random Forest", y_pred_floresta)]:
    print(f"\n{nome}")
    print("Acurácia:", accuracy_score(y_test, y_pred))
    print("Precisão:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1-Score:", f1_score(y_test, y_pred))



Árvore
Acurácia: 0.9290755919854281
Precisão: 0.952
Recall: 0.7555555555555555
F1-Score: 0.8424778761061947

Random Forest
Acurácia: 0.9982923497267759
Precisão: 0.997275204359673
Recall: 0.9959183673469387
F1-Score: 0.9965963240299524


# Relatório Executivo – Comparação de Modelos (Sprint 4)
## 1. Visão Geral

Esta sprint teve como objetivo a construção, análise e comparação de diferentes modelos de aprendizado de máquina aplicados ao problema de previsão de falta de estoque crítico, utilizando dados históricos de materiais hospitalares. Foram implementadas abordagens tanto de regressão quanto de classificação, avaliando desempenho, interpretabilidade e adequação ao contexto operacional.

## 2. Modelos Avaliados
### 2.1 Regressão Linear Simples e Múltipla


Principais variáveis explicativas: Custo_Total, Custo_Unitario, e dummies de Material

Objetivo: Estimar o consumo mensal com base em custos e tipo de material.

#### Resumo:

Os modelos apresentaram desempenho consistente, sendo a Regressão Múltipla mais robusta ao incorporar variáveis categóricas (materiais).

Coeficientes mostraram influência negativa para materiais como Gaze Estéril e Álcool 70%, sugerindo maior consumo relativo (menor estoque disponível).

### 2.2 Regressões Regularizadas (Ridge e Lasso)

<table> 
    <thead>
        <tr>
            <th>Modelo</th>
            <th>R²</th>
            <th>RMSE</th>
            <th>Interpretação</th>
        </tr>
    </thead> 
    <tbody> 
        <tr>
            <td><b>Ridge</b></td>
            <td>0.636</td>
            <td>94.30</td>
            <td>Reduziu variância e suavizou coeficientes extremos, porém com leve perda de explicação.</td>
        </tr> 
        <tr>
            <td><b>Lasso</b></td>
            <td>—</td>
            <td>—</td>
            <td>Reduziu o número de variáveis para 8, eliminando colunas com baixa relevância.</td>
        </tr> 
    </tbody> 
</table>

#### Análise:

* A Ridge Regression foi eficiente para reduzir overfitting e estabilizar coeficientes.

* A Lasso Regression destacou as variáveis mais relevantes (Dipirona, Paracetamol, Algodão, Álcool 70%), auxiliando na interpretação e simplificação do modelo.

### 2.3 Regressão Polinomial

* R²: 0.9975

* RMSE: 7.81

#### Análise:

A regressão polinomial de grau 2 apresentou ganho expressivo de desempenho, indicando relação não-linear entre custo e estoque.

Apesar da alta precisão, há risco de overfitting — o modelo se ajusta muito bem aos dados de treino, podendo perder generalização.

### 2.4 Modelos de Classificação
* K-Nearest Neighbors (KNN)

* Acurácia: 0.9946

* Precisão: 0.9900

* Recall: 0.9887

* F1-Score: 0.9893

#### Matriz de Confusão:
<table> 
    <thead>
        <tr>
            <th></th>
            <th>Predito 0</th>
            <th>Predito 1</th>
        </tr>
    </thead> 
    <tbody> 
        <tr>
            <th>Real 0</th>
            <td>6557</td>
            <td>22</td>
        </tr> 
        <tr>
            <th>Real 1</th>
            <td>25</td>
            <td>2180</td>
        </tr> 
    </tbody> 
</table>

#### Análise:
O modelo KNN apresentou excelente desempenho, com baixo número de falsos positivos e falsos negativos. O valor de k foi ajustado de forma a equilibrar suavização e sensibilidade, oferecendo boa performance para detecção de risco de desabastecimento.

### Regressão Logística

<table> 
    <thead>
        <tr>
            <th>Métrica</th>
            <th>Valor</th>
        </tr>
    </thead> 
    <tbody> 
        <tr>
            <td>Acurácia</td>
            <td>0.9967</td>
        </tr> 
        <tr>
            <td>Precisão</td>
            <td>0.9910</td>
        </tr> 
        <tr>
            <td>Recall</td>
            <td>0.9959</td>
        </tr> 
        <tr>
            <td>F1-Score</td>
            <td>0.9934</td>
        </tr> 
    </tbody> 
</table>

#### Interpretação dos Coeficientes:
Os sinais dos coeficientes indicaram quais materiais têm maior probabilidade associada a eventos críticos de estoque.
Materiais com coeficientes negativos (ex.: Gaze Estéril, Álcool 70%) estão mais propensos a quedas de estoque, o que pode orientar políticas de reposição.

### Comparativo KNN vs. Regressão Logística
<table> 
    <thead>
        <tr>
            <th>Modelo</th>
            <th>Acurácia</th>
            <th>Precisão</th>
            <th>Recall</th>
            <th>F1-Score</th>
        </tr>
    </thead> 
    <tbody> 
        <tr>
            <td><b>KNN</b></td>
            <td>0.9946</td>
            <td>0.9900</td>
            <td>0.9887</td>
            <td>0.9893</td>
        </tr> 
        <tr>
            <td><b>Regressão Logística</b></td>
            <td>0.9967</td>
            <td>0.9910</td>
            <td>0.9959</td>
            <td>0.9934</td>
        </tr> 
    </tbody> 
</table>

#### Conclusão:
A Regressão Logística teve desempenho levemente superior e oferece melhor interpretabilidade, tornando-a mais adequada para ambientes hospitalares que exigem transparência em decisões.

### 2.5 Árvores de Decisão e Random Forest
<table> 
    <thead>
        <tr>
            <th>Modelo</th>
            <th>Acurácia</th>
            <th>Precisão</th>
            <th>Recall</th>
            <th>F1-Score</th>
        </tr>
    </thead> 
    <tbody> 
        <tr>
            <td><b>Árvore de Decisão</b></td>
            <td>0.9291</td>
            <td>0.952</td>
            <td>0.756</td>
            <td>0.842</td>
        </tr> 
        <tr>
            <td><b>Random Forest</b></td>
            <td>0.9983</td>
            <td>0.9973</td>
            <td>0.9959</td>
            <td>0.9966</td>
        </tr> 
    </tbody> 
</table>

#### Análise:

A Árvore de Decisão é facilmente interpretável, mas sofreu com sobreajuste e menor recall.

A Random Forest superou todos os classificadores, com altíssimo desempenho e boa generalização, tornando-se o modelo mais equilibrado entre precisão e robustez.

## 3. Conclusões e Recomendação Final
<table> 
    <thead>
        <tr>
            <th>Categoria</th>
            <th>Melhor Modelo</th>
            <th>Justificativa</th>
        </tr>
    </thead> 
    <tbody> 
        <tr>
            <td>Regressão Contínua</td>
            <td><b>Regressão Polinomial</b></td>
            <td>Captura relação não-linear e apresenta R² próximo de 1.</td>
        </tr> 
        <tr>
            <td>Classificação Binária</td>
            <td><b>Random Forest</b></td>
            <td>Excelente equilíbrio entre precisão, recall e robustez.</td>
        </tr> 
        <tr>
            <td>Modelo Explicável</td>
            <td><b>Regressão Logística</b></td>
            <td>Fornece coeficientes interpretáveis úteis para decisões de estoque.</td>
        </tr> 
    </tbody> 
</table>



#### Recomendação Final:
A Random Forest é a abordagem mais indicada para a detecção de risco de falta de estoque crítico, devido à sua alta acurácia e capacidade de generalização.
No entanto, a Regressão Logística deve ser mantida como modelo de apoio interpretativo, permitindo explicar as variáveis que mais contribuem para o evento de desabastecimento.


## Resumo Geral das Métricas

<table> 
    <thead>
        <tr>
            <th>Modelo</th>
            <th>Tipo</th>
            <th>Acurácia</th>
            <th>R²</th>
            <th>RMSE</th
            ><th>F1-Score</th>
        </tr>
    </thead> 
    <tbody> 
        <tr>
            <td>Regressão Simples</td>
            <td>Regressão</td>
            <td>—</td>
            <td>0.63</td>
            <td>94.30</td>
            <td>—</td>
        </tr> 
        <tr>
            <td>Ridge</td>
            <td>Regressão</td>
            <td>—</td>
            <td>0.64</td>
            <td>94.30</td>
            <td>—</td>
        </tr> 
        <tr>
            <td>Lasso</td>
            <td>Regressão</td>
            <td>—</td>
            <td>—</td>
            <td>—</td>
            <td>—</td>
        </tr> 
        <tr>
            <td>Polinomial</td>
            <td>Regressão</td>
            <td>—</td>
            <td><b>0.9975</b></td>
            <td><b>7.81</b></td>
            <td>—</td>
        </tr> 
        <tr>
            <td>KNN</td>
            <td>Classificação</td>
            <td>0.9946</td>
            <td>—</td>
            <td>—</td>
            <td>0.9893</td>
        </tr> 
        <tr>
            <td>Regressão Logística</td>
            <td>Classificação</td>
            <td>0.9967</td>
            <td>—</td>
            <td>—</td>
            <td>0.9934</td>
        </tr> 
        <tr>
            <td>Árvore de Decisão</td>
            <td>Classificação</td>
            <td>0.9291</td>
            <td>—</td>
            <td>—</td>
            <td>0.8425</td>
        </tr> 
        <tr>
            <td>Random Forest</td>
            <td>Classificação</td>
            <td><b>0.9983</b></td>
            <td>—</td>
            <td>—</td>
            <td><b>0.9966</b></td>
        </tr>
    </tbody> 
</table>